In [155]:
import tensorflow as tf
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [156]:
def convert_folder_images(folder_name):

    # specify the folder path
    folder_path = f"alphabet_data/{folder_name}"

    # initialize an empty list to store the image arrays
    image_arrays = []

    # loop through each file in the folder
    for filename in os.listdir(folder_path):
        # check that the file is an image file (e.g. JPEG, PNG)
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # construct the full file path
            file_path = os.path.join(folder_path, filename)
            # read the image file as a numpy array
            img_array = cv2.imread(file_path)

            #gray the images
            # img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY) # from 3 channels to 1 channel
            # # coverting to gray scale gets rid of one dimension, so we add it back in, this time with 1 channel
            # # Add an extra dimension to the grayscale image
            # img_array = np.expand_dims(img_array, axis=-1)

            # add the image array to the list
            image_arrays.append(img_array)

    # convert the list of image arrays to a numpy array
    image_arrays = np.array(image_arrays)
    return image_arrays

In [157]:
def get_all_images_and_labels():
    a = convert_folder_images("a")
    # make labels for all photos based on how many images are in the folder
    # the folder names gives you the label for each folder of images

    b = convert_folder_images("b")
    c = convert_folder_images("c")
    d = convert_folder_images("d")
    e = convert_folder_images("e")

    labels_a = np.full(len(a),"a")
    labels_b = np.full(len(b),"b")
    labels_c = np.full(len(c),"c")
    labels_d = np.full(len(d),"d")
    labels_e = np.full(len(e),"e")
    # print(labels_e.shape)
    images = np.vstack((a,b,c,d,e))/255 # divide by 255 for normalization
    # labels = np.append(labels_a, (labels_b, labels_c, labels_d, labels_e))

    # Convert string labels to numerical labels using LabelEncoder
    label_encoder = LabelEncoder()
    # concatenate all the labels together
    labels = np.concatenate(
        (labels_a, labels_b, labels_c, labels_d, labels_e), axis=0)
    labels = label_encoder.fit_transform(labels)


    return images,labels

In [158]:
# make training, validation, and testing sets
images,labels = get_all_images_and_labels()
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=2)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
# images[5][:][:][:].shape
# plt.imshow(one_image)

In [159]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224,3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])
# type(model)

In [160]:

# configure the learning process
model.compile(loss='sparse_categorical_crossentropy',
              # use sparse cross entropy since your data is integer encoded, NOT hot-encoded (0s and 1s)
              optimizer='adam',
              metrics=['accuracy']),
# print(tf.config.list_physical_devices('GPU'))

(None,)

In [161]:
# Check if GPU is available and set device accordingly
if tf.test.gpu_device_name():
    print('GPU found')
    device_name = tf.test.gpu_device_name()
else:
    print("No GPU found, using CPU instead")
    device_name = '/cpu:0'

No GPU found, using CPU instead


In [162]:
# configure the learning process
model.compile(loss='sparse_categorical_crossentropy',
              # use sparse cross entropy since your data is integer encoded, NOT hot-encoded (0s and 1s)
              optimizer='adam',
              metrics=['accuracy']),
# print(tf.config.list_physical_devices('GPU'))

(None,)

In [145]:
@tf.autograph.experimental.do_not_convert
def fit_model():
    model.fit(x_train,
              y_train,
              batch_size=50,
              epochs=10,
              validation_data=(x_valid, y_valid), verbose=1,
              callbacks=None)
# test the accuracy of model with testing ste
def test_set_eval():

    # Evaluate the model on test set
    score = model.evaluate(x_test, y_test, verbose=0)
    # Print test accuracy
    print('\n', 'Test accuracy:', score[1])
    # model.predict(x_test)
# export the model to the "Model" folder
def export_model():
    want_to_export = input("export model? ('true' or 'false'")
    if want_to_export:
        model_path = 'Model/keras_model.h5'
        tf.keras.models.save_model(model, model_path)

In [146]:
# Create a TensorFlow session and set it to use the specified device
with tf.device(device_name):
    # train the model
    fit_model()
    test_set_eval()

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4/4 [==============================] - ETA: 0s - loss: 5.7236 - accuracy: 0.3313WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001E8DD313708> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4/4 [==============================] - 6s 1s/step - loss: 5.7236 - accuracy: 0.3313 - val_loss: 1.9320 - val_accuracy: 0.7500
Epoch 2/10
4/4 [==============================] - 5s 1s/step - loss: 3.5114 - accuracy: 0.6000 - val_loss: 0.2435 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 5s 1s/step - loss: 0.5622 - accuracy: 0.8438 - val_loss: 0.3080 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 5s 1s/step - loss: 0.1047 - accuracy: 0.9688 - val_loss: 0.2318 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 5s 1s/step - loss: 0.0149 - accuracy: 1.0000 - val_loss: 0.1402 - val_accuracy: 1.0000
Epoch 6/10
4/4 

In [147]:
test_set_eval()


 Test accuracy: 1.0


In [148]:
export_model()


keras.engine.sequential.Sequential